### Show an example Dash app using components in dash_df_components

In [1]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import pandas as pd
import numpy as np
import traceback

from dashgrid import dash_df_components as ddfc
dgc = ddfc.dgc


## Example of a Dash App using VariableRowDiv

### Step 00: Define a logger that all components can use

In [2]:
logger = dgc.init_root_logger('logfile.log','INFO')

### Step 01: Create a title div for our app
1. Create a line of markdown for the top title
2. Create a multi-line markdown using numbered bullets

In [3]:
tm1 = ddfc.MarkdownLine('Dash DataFrame App',text_size=1)
tm_text2 = '''
            1. This app displays Pandas DataFrames, Plotly Graphs and other html components 
            2. like file inputs, dropdown lists, and text input fields, and general divs.

             '''
tm2 = ddfc.MarkdownLine(tm_text2,text_align='left')
title_comp = ddfc.MarkdownDiv('mdiv1',[tm1,tm2])


### Step 02: Create some intial data for our app
1. Create a DataFrame
2. Create a table div
3. Create an xy graph div

In [4]:
x = np.arange(1,10,.1).round(4)
df = pd.DataFrame({'x':x,'y':(x + np.random.normal(size=len(x))*.1).round(4)})

table_example = ddfc.TableInput(
    'init_table',initial_data=df,title='Initial Table')
xy_example = ddfc.XYGraphSimple(
    'init_graph',initial_data=df,x_column='x',title='Initial Graph')  


### Step 03: Create an input textbox so that you can:
1. Enter a floating point number
2. See a second graph change, which displays a line that is:
 * a multiple of the line in the first graph
 * with some additional randomness

In [5]:
input_div = dgc.DivComponent('input_div',initial_children="Enter a number")
input_comp = ddfc.TextBoxInput('input_box',init_value=1,input_type='number')

### Step 04: Create a callback that 
#### First: excepts a dictionary of data like:
1. Pandas Dataframes
2. String values

####  Second: outputs 2 new rows, where each row contains:
1. a pandas DataFrame that will be used by ```dash_df_components.TableInput```
2. the same pandas DataFrame that will be used by ```dash_df_components.XYGraphSimple```


In [6]:
# your input argument is a single dictionary or data, like a pandas DataFrame
# your callback method should return a 2 dimensional array of data types that coincide
#   with the data types that your repeating components need.
# In this case, we are repeating a table an a graph.  Both of these component need a 
#    pandas DataFrame. 
def my_callback(input_dict):   
    df = input_dict['init_table']
    try:
        if 'x' not in df.columns.values:
            return [[None,None]]
        x = df.x
        scale_y = input_dict['input_box']
        if scale_y is None:
            scale_y = 1
        scale_y = float(str(scale_y))
        y2 = x * scale_y + np.random.normal(size=len(x))*.1
        df2 = pd.DataFrame({'x':x,'y':y2})    
        df3 = pd.DataFrame({'x':x,'y':y2*-1}) 
        # return a 2 dimensional array, where each dimension has 2 DataFrames
        return [[df2,df2],[df3,df3]]
    except Exception as e:
        traceback.print_exc()
        return [[None,None],[None,None]]


### Step 05: Create a VariableRowDiv, a class that allows you to create a variable number of repeating rows of Tables, Graphs, Maps, and many other graphical elements found in Dash.

#### The arguments are as follows:
* arg0: the id of the VariableRowDiv
* arg1: a list of class instances from the component library dash_df_components
* arg2: your callback that creates data for each repeating row
* arg3: a css grid layout template for each repeating row
* arg4: a logger

In [7]:
v = ddfc.VariableRowDiv('my_app',
            [table_example,input_comp],
            my_callback,
            [table_example,xy_example],
            '1fr 1fr',
            logger=logger)

ac = [title_comp,input_div,input_comp,table_example,xy_example] + v.final_components
gc = ['1fr','1fr 1fr','1fr 1fr'] + v.final_layout

app = dgc.make_app(ac,grid_template_columns_list=gc)    
app.run_server(host='127.0.0.1',port=8500)    


2020-01-26 22:39:49,318 - root - INFO - StoreComponent self.output_data_tuple ('store_comp_my_app', 'data')


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-26 22:39:49,359 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-26 22:39:57,949 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 22:39:57] "GET / HTTP/1.1" 200 -
2020-01-26 22:39:57,996 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 22:39:57] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 22:39:58,003 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 22:39:58] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 22:39:58,013 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 22:39:58] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-26 22:39:58,033 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 22:39:58] "GET /_dash-component-suites/dash_table/bundle.js?v=4.0.1&m=1574289295 HTTP/1.1" 200 -
2020-01-26 22:39:58,038 - werkzeug - INFO - 127.0.0.1 - - [26/Jan/2020 2